<a href="https://colab.research.google.com/github/pri2k/BTP/blob/main/Exploringlib/classification2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Uploading files

In [1]:
# Step 1: Upload the file
from google.colab import files
uploaded = files.upload()

# Get the filename
filename = list(uploaded.keys())[0]

# Read and join all paragraphs as one big text
with open(filename, 'r', encoding='utf-8') as f:
    content = f.read()

# Step 2: Split lines and filter out '---------' and blank lines
lines = content.strip().splitlines()
filtered_lines = [line.strip() for line in lines if not line.strip().startswith('-') and line.strip() != '']

# Step 3: Join into paragraphs and flatten
combined_text = ' '.join(filtered_lines)


# lines = content.strip().splitlines()
# filtered_lines = [line.strip() for line in lines if not line.strip().startswith('-') and line.strip() != '']
# combined_text = ' '.join(filtered_lines)


# # Split on paragraph breaks (double newline), then join with space

# paragraphs = content.strip().split('\n\n')
# combined_text = ' '.join(paragraph.strip().replace('\n', ' ') for paragraph in paragraphs)

# print(f" Combined {len(paragraphs)} paragraphs into one block of {len(combined_text.split())} words.")


Saving catalan_independence to catalan_independence
Saving harvey_weinstein to harvey_weinstein
Saving demonetization_9 to demonetization_9
Saving ball_tampering to ball_tampering


Installing requirements (you only need to run it one time when running the code for the first time)

In [2]:
!pip install rake-nltk yake
# !pip uninstall -y transformers
!pip install transformers --upgrade

!pip install --upgrade huggingface_hub
!pip install --upgrade keybert sentence-transformers

import nltk
nltk.download('stopwords')
nltk.download('punkt_tab')


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.2/60.2 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 356.9/356.9 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 108.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 81.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

import and extractors defining

In [8]:
from transformers import AutoTokenizer, AutoModel
!git lfs install
!git clone https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2

Git LFS initialized.
fatal: destination path 'all-MiniLM-L6-v2' already exists and is not an empty directory.


In [12]:
# RAKE
from rake_nltk import Rake

# YAKE
import yake

# KeyBERT
from keybert import KeyBERT

# Initialize models
rake = Rake()
kw_extractor_yake = yake.KeywordExtractor(lan="en", top=10)
kw_model = KeyBERT(model="sentence-transformers/all-MiniLM-L6-v2")


tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

functions for each extractor

In [23]:
# def extract_keywords_rake(text, top_n=100):
#     rake.extract_keywords_from_text(text)
#     return rake.get_ranked_phrases_with_scores()[:top_n]

def extract_keywords_rake(text, top_n=100, min_words=1, max_words=1):
    rake.extract_keywords_from_text(text)
    ranked_phrases = rake.get_ranked_phrases_with_scores()

    seen = set()
    filtered = []

    for score, phrase in ranked_phrases:
        phrase_clean = phrase.strip().lower()
        word_count = len(phrase_clean.split())
        if phrase_clean not in seen and min_words <= word_count <= max_words:
            filtered.append((score, phrase))
            seen.add(phrase_clean)
        if len(filtered) >= top_n:
            break

    return filtered


def extract_keywords_yake(text):
    return kw_extractor_yake.extract_keywords(text)

def extract_keywords_keybert(text):
    return kw_model.extract_keywords(
        text,
        top_n=100,
        stop_words='english',
        keyphrase_ngram_range=(1, 2)
    )


Calling the functions and printing

In [20]:
print("\n==============================")
print(" Extracting keywords from entire article collection\n")

print("---  RAKE Keywords ---")
for score, kw in extract_keywords_rake(combined_text):
    print(f"{kw} ({score:.2f})")

print("\n---  YAKE Keywords ---")
for kw, score in extract_keywords_yake(combined_text):
    print(f"{kw} ({score:.6f})")

print("\n---  KeyBERT Keywords ---")
for kw, score in extract_keywords_keybert(combined_text):
    print(f"{kw} ({score:.4f})")



 Extracting keywords from entire article collection

---  RAKE Keywords ---
ciutadans (7.67)
image (6.14)
epa (5.50)
warrants (5.00)
orders (4.67)
vera (4.50)
understand (4.25)
women (4.00)
mep (4.00)
clients (4.00)
‘ (3.93)
ponsati (3.83)
world (3.78)
25 (3.78)
,” (3.76)
foreign (3.75)
behind (3.75)
backed (3.75)
.” (3.72)
five (3.71)
oriol (3.70)
since (3.68)
action (3.67)
– (3.64)
” (3.62)
el (3.60)
last (3.58)
immediately (3.57)
says (3.56)
," (3.55)
minister (3.52)
using (3.50)
slideshow (3.50)
seeks (3.50)
positions (3.50)
news (3.50)
landmark (3.50)
imprisoned (3.50)
dismissed (3.50)
president (3.47)
issued (3.42)
among (3.42)
spokesman (3.40)
requested (3.40)
catalan (3.35)
jailed (3.35)
react (3.33)
morning (3.33)
legitimate (3.33)
honor (3.33)
fleeing (3.33)
find (3.33)
campaigning (3.33)
advocates (3.33)
2006 (3.33)
1714 (3.33)
dictatorship (3.29)
rajoy (3.28)
month (3.27)
secretary (3.25)
never (3.20)
maragall (3.20)
alonso (3.20)
new (3.19)
european (3.19)
declared (3.18)

In [21]:
rake_keywords = extract_keywords_rake(combined_text)  # (score, keyword)
yake_keywords = extract_keywords_yake(combined_text)  # (keyword, score)
keybert_keywords = extract_keywords_keybert(combined_text)  # (keyword, score)
print("Succesfully stored")


Succesfully stored


Downloading the extracted keywords


In [22]:
import pandas as pd

# Convert to DataFrames
rake_df = pd.DataFrame(rake_keywords, columns=["Score", "Keyword"])
yake_df = pd.DataFrame(yake_keywords, columns=["Keyword", "Score"])
keybert_df = pd.DataFrame(keybert_keywords, columns=["Keyword", "Score"])

# Save to CSV files
rake_df.to_csv("Dem_rake_keywords.csv", index=False)
yake_df.to_csv("Dem_yake_keywords.csv", index=False)
keybert_df.to_csv("Dem_keybert_keywords.csv", index=False)

print("Keywords saved to CSV files.")

from google.colab import files

files.download("Dem_rake_keywords.csv")
files.download("Dem_yake_keywords.csv")
files.download("Dem_keybert_keywords.csv")



Keywords saved to CSV files.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

CLASSIFY WHETHER TARGET OR NOT

In [17]:
def classify_keywords(keyword_list, method_keywords, threshold, method_name):
    if method_name == "RAKE":
        # RAKE gives (score, phrase), so we reverse it
        kw_dict = {kw.lower(): score for score, kw in method_keywords}
    else:
        kw_dict = {kw.lower(): score for kw, score in method_keywords}

    results = {}

    for keyword in keyword_list:
        keyword_lower = keyword.lower()
        found = False

        for phrase, score in kw_dict.items():
            if keyword_lower in phrase:
                # print(f"[DEBUG] {keyword_lower} matched phrase '{phrase}' with score {score}")
                if method_name in ["RAKE", "KeyBERT"]:
                    if score >= threshold:
                        found = True
                        break
                else:  # YAKE: lower is better
                    if round(score, 6) < round(threshold, 6):
                        found = True
                        break

        results[keyword] = found

    return results


For every new article make sure to change the keyword to test and the threshold values.

In [18]:
keywords_to_test = ["Catalan independence"]
raketest=["susana vera “ falling apart ” puigdemont appeared","mobile world congress","former parliament speaker carme forcadell"]
yaketest=["Spain","Catalan independence"]
keyberttest=["catalonia want independence","regional election catalonia","catalonians voted","spain called elections"]

rake_threshold = 30
yake_threshold = 0.000277  # e.g. raw score, lower means more important
keybert_threshold = 0.63

rakewords= classify_keywords(raketest, rake_keywords, rake_threshold, "RAKE")
yakewords= classify_keywords(yaketest, yake_keywords, yake_threshold, "YAKE")
keybertwords= classify_keywords(keyberttest, keybert_keywords, keybert_threshold, "KeyBERT")

print("Rake classification")
for word, is_target in rakewords.items():
    print(f"{word} → {is_target}")
print("\n")

print("Yake classification")
for word, is_target in yakewords.items():
    print(f"{word} → {is_target}")
print("\n")

print("Keybert classification")
for word, is_target in keybertwords.items():
    print(f"{word} → {is_target}")


Rake classification
susana vera “ falling apart ” puigdemont appeared → False
mobile world congress → False
former parliament speaker carme forcadell → False


Yake classification
Spain → False
Catalan independence → True


Keybert classification
catalonia want independence → True
regional election catalonia → True
catalonians voted → False
spain called elections → False
